---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [1]:
import pandas as pd
import numpy as np

## load the data to process
def load_data():
    # load the train set
    train_set = pd.read_csv('train.csv', encoding = "ISO-8859-1", low_memory=False)
    # load the submit set
    submit_set = pd.read_csv('test.csv', encoding = "ISO-8859-1", low_memory=False)
    # drop out NaN compliance rows
    train_set.dropna(axis = 0, how = 'any', subset = ['compliance'], inplace = True)
    # this is the target column for training
    y = train_set['compliance']
    # these are columns inaccessible in real application
    unpractical_columns = ['payment_amount', 'payment_date', 'payment_status', 'balance_due', \
                           'collection_status', 'compliance', 'compliance_detail']
    # exclude these columns to avoid leakage
    train_set.drop(unpractical_columns, axis = 1, inplace = True)
    # concat train and submit set to avoid dismatch in dummy variables
    X = pd.concat((train_set, submit_set))
    return X, y

In [2]:
def feature_drop(X):
    # inspector_name has many categories, tedious
    #print(X['inspector_name'].unique())
    X.drop(['inspector_name'], axis = 1, inplace = True)

    # violator_name has many categories, tedious
    #print(X['violator_name'].unique())
    X.drop(['violator_name'], axis = 1, inplace = True)

    #print(X['violation_street_name'].unique())
    #print(X['violation_street_number'].head())
    X.drop(['violation_street_name'], axis = 1, inplace = True)
    X.drop(['violation_street_number'], axis = 1, inplace = True)

    # violation_zip_code is basically NaN
    #print(X['violation_zip_code'])
    X.drop(['violation_zip_code'], axis = 1, inplace = True)

    # mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country
    # address can be replaced by lat/lon coordinates
    #print(X['mailing_address_str_number'].unique())
    #print(X['mailing_address_str_name'].unique())
    #print(X['city'].unique())
    #print(X['state'].unique())
    #print(X['zip_code'].unique())
    #print(X['non_us_str_code'].unique())
    #print(X['country'].unique())

    X.drop(['mailing_address_str_number'], axis = 1, inplace = True)
    X.drop(['mailing_address_str_name'], axis = 1, inplace = True)
    X.drop(['city'], axis = 1, inplace = True)
    X.drop(['state'], axis = 1, inplace = True)
    X.drop(['zip_code'], axis = 1, inplace = True)
    X.drop(['non_us_str_code'], axis = 1, inplace = True)
    X.drop(['country'], axis = 1, inplace = True)

    # ticket_issued_date, string to be replaced by datetime ? drop
    #dt = X['ticket_issued_date'].apply(string2datetime)
    #X = X.join(dt)
    #print(dt.head())
    X.drop(['ticket_issued_date'], axis = 1, inplace = True)
    
    #print(X.head())

    # hearing_date, string to be replaced by datetime ? drop
    #dt = X['hearing_date'].apply(string2datetime)
    #X = X.join(dt)
    X.drop(['hearing_date'], axis = 1, inplace = True)   

    # violation_description is redundant given violation_code
    X.drop(['violation_description'], axis = 1, inplace = True)

    # grafitti_status is basically nan
    #print(X['grafitti_status'].unique())
    X.drop(['grafitti_status'], axis = 1, inplace = True)
    return X

In [3]:
def vio_code(X):
    # the violation_code has quite a few categories, but maybe useful as dummies
    #print(X['violation_code'].unique())
    X = pd.concat([X, pd.get_dummies(X['violation_code']).astype(np.int8)], axis=1)
    X.drop(['violation_code'], axis = 1, inplace = True)
    return X

In [4]:
def agency_name(X):
    # agency_name has few categories, maybe useful as dummies
    #print(X['agency_name'].unique())
    X = pd.concat([X, pd.get_dummies(X['agency_name']).astype(np.int8)], axis=1)
    X.drop(['agency_name'], axis = 1, inplace = True)
    return X

In [5]:
def latlon(X):        
    # ticket_id to address
    addresses = pd.read_csv('addresses.csv', encoding = "ISO-8859-1")
    addresses.set_index('ticket_id', inplace = True)
    # address to lat/lon
    latlons = pd.read_csv('latlons.csv', encoding = "ISO-8859-1")
    latlons.set_index('address', inplace = True)
    
    # violation_street_name/violation_street_number can be replaced by lat/lon coordinates
    #print(X['violation_street_name'].unique().size)
    #print(X['violation_street_number'].unique().size)
    #print(addresses.head())
    #print(latlons.head())
    address = X['ticket_id'].apply(lambda x : addresses.loc[x])
    #print(address.head())

    lat = address.applymap(lambda x : latlons.loc[x]['lat'])
    lon = address.applymap(lambda x : latlons.loc[x]['lon'])
    lat.columns = ['lat']
    lon.columns = ['lon']
    #print(lat.head())
    #print(lon.head())
    X = pd.concat([X, lat, lon], axis=1)  
    return X

In [6]:
def disp_and_na(X):
    # disposition has few categories, maybe useful as dummies
    #print(X['disposition'].unique())   
    X = pd.concat([X, pd.get_dummies(X['disposition']).astype(np.int8)], axis=1)
    X.drop(['disposition'], axis = 1, inplace = True)
    # fill the nan with mean
    X.fillna(X.mean(), inplace = True)
    return X

In [18]:
## train models
def train(X, y):
    # using random forest as baseline
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.model_selection import GridSearchCV
    from sklearn.metrics import roc_auc_score
    grid_values = {'n_estimators': [50, 100, 400, 800]}
    clf = RandomForestClassifier(random_state = 0, n_jobs = 4)
    grid_clf = GridSearchCV(clf, param_grid = grid_values, scoring = 'roc_auc', cv = 3)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)   
    
    grid_clf.fit(X_train, y_train)
    
    print(grid_clf.cv_results_['mean_test_score'])   
    print("test score:{:.2f}".format(roc_auc_score(grid_clf.predict_proba(X_test), y_test)))
    return grid_clf

In [9]:
X, y = load_data()

In [10]:
X = feature_drop(X)

In [11]:
X = vio_code(X)

In [12]:
X = agency_name(X)

In [13]:
X = latlon(X)

In [14]:
X = disp_and_na(X)

In [15]:
print(X.head())

   ticket_id  fine_amount  admin_fee  state_fee  late_fee  discount_amount  \
0      22056        250.0       20.0       10.0      25.0              0.0   
1      27586        750.0       20.0       10.0      75.0              0.0   
5      22046        250.0       20.0       10.0      25.0              0.0   
6      18738        750.0       20.0       10.0      75.0              0.0   
7      18735        100.0       20.0       10.0      10.0              0.0   

   clean_up_cost  judgment_amount  19410901  19420901  \
0            0.0            305.0         0         0   
1            0.0            855.0         0         0   
5            0.0            305.0         0         0   
6            0.0            855.0         0         0   
7            0.0            140.0         0         0   

             ...                   lat        lon  \
0            ...             42.390729 -83.124268   
1            ...             42.326937 -83.135118   
5            ...             

In [16]:
X_train_set = X[:len(y.index)]
X_submit_set = X[len(y.index):]

In [17]:
grid_clf = train(X_train_set, y)

[ 0.75198432  0.79345634  0.79922181  0.80049898]
test score:0.80


In [20]:
y_submit = [proba[1] for proba in grid_clf.predict_proba(X_submit_set)]

In [1]:
#print(y_submit)

In [2]:
#print(X_submit_set['ticket_id'].tolist())

In [1]:
def blight_model():
    submit = pd.Series(data = y_submit, index = X_submit_set['ticket_id'], name = 'compliance')
    return submit